This query is for selecting the vintages priced < 20 €    
They seem insteresting to highlight just because they got good rating and are cheap

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import streamlit as st
# Connect to SQLite database
conn = sqlite3.connect('../data/vivino.db')
# Create a cursor object
cur = conn.cursor()
cur.execute("""
    SELECT
        v.id,
        v.ratings_average,
        v.ratings_count,
        v.name,
        v.price_euros
        
    FROM
        vintages v
    WHERE
        v.price_euros < 20
    ORDER BY
        v.ratings_average DESC,
        v.ratings_count DESC
    LIMIT 2;
""")
# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['id','ratings_average', 'ratings_count', 'name', 'price_euros'])
print(df)
# Close the database connection
conn.close()

In [ ]:
    SELECT
        v.id,
        v.ratings_average,
        v.ratings_count,
        v.name,
        v.price_euros
        
    FROM
        vintages v
    WHERE
        v.price_euros < 20
    ORDER BY
        v.ratings_average DESC,
        v.ratings_count DESC
    LIMIT 2;

This query is for selecting wines with price between 20 and 30 euros, avg rating >= 4.4, rating_count > 1000.     
The wines are not N.V. (non-vintage: blend of multiple years)

In [54]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connect to the SQLite database
conn = sqlite3.connect(r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\vivino.db')

# Execute the query and fetch data
query = """
SELECT v.id, v.name AS vintage_name, v.ratings_average, v.year, v.price_euros, v.ratings_count
FROM vintages AS v
JOIN wines AS w ON v.wine_id = w.id
JOIN regions AS r ON w.region_id = r.id
WHERE v.ratings_average >= 4.4 AND v.price_euros BETWEEN 20 AND 50 AND v.ratings_count > 1000 AND v.year != 'N.V.'
ORDER BY v.year DESC
LIMIT 8;
"""
result = conn.execute(query).fetchall()

# Close the database connection
conn.close()

# Convert the result to a Pandas DataFrame
columns = ['id', 'vintage_name', 'ratings_average', 'year', 'price_euros', 'ratings_count']
df = pd.DataFrame(result, columns=columns)

print(df)

          id                                       vintage_name  \
0  162964134                     Pago de Carraovejas Tinto 2020   
1  159319855                                 Rutini Malbec 2019   
2  160549459               Guerrieri Guerriero della Terra 2019   
3  156098276  San Marzano 60 Sessantanni Old Vines Primitivo...   
4  156113777          Matarromera Ribera del Duero Reserva 2018   
5  156132702   Tommasi Amarone della Valpolicella Classico 2018   
6  150450962          Matarromera Ribera del Duero Reserva 2017   
7  152069724                           Gitana Lupi Rezerva 2017   
8   14794438             Bodegas Roda Roda I Reserva Rioja 2015   
9    2667186  Château Suduiraut Sauternes (Premier Grand Cru...   

   ratings_average  year  price_euros  ratings_count  
0              4.5  2020        42.95           2129  
1              4.4  2019        25.99           2567  
2              4.4  2019        27.40           1803  
3              4.4  2018        24.75     

Merged queries and output df and csv    
The query returns 10 vintages to highlight to increase sales.

In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\vivino.db')

# Execute the combined query and fetch data
combined_query = """
SELECT v.id, v.name AS vintage_name, v.ratings_average, v.year, v.price_euros, v.ratings_count, w.url
FROM vintages AS v
JOIN wines AS w ON v.wine_id = w.id
JOIN regions AS r ON w.region_id = r.id
WHERE (v.ratings_average >= 4.4 AND v.price_euros < 20) OR (v.ratings_average >= 4.4 AND v.price_euros BETWEEN 20 AND 50 AND v.ratings_count > 1000 AND v.year != 'N.V.')
ORDER BY v.year DESC
LIMIT 10;
"""
result = conn.execute(combined_query).fetchall()

# Close the database connection
conn.close()

# Convert the result to a Pandas DataFrame
columns = ['id', 'vintage_name', 'ratings_average', 'year', 'price_euros', 'ratings_count', 'url']
df = pd.DataFrame(result, columns=columns)
print(df.info())
# Save the DataFrame to a CSV file in the "data" directory
csv_file_path = r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\wines_to_highlight.csv'
df.to_csv(csv_file_path, index=False)

# Print the DataFrame
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               10 non-null     int64  
 1   vintage_name     10 non-null     object 
 2   ratings_average  10 non-null     float64
 3   year             10 non-null     int64  
 4   price_euros      10 non-null     float64
 5   ratings_count    10 non-null     int64  
 6   url              10 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 692.0+ bytes
None
          id                                       vintage_name  \
0  167945269               Cignomoro Primitivo di Manduria 2021   
1  162964134                     Pago de Carraovejas Tinto 2020   
2  159319855                                 Rutini Malbec 2019   
3  160549459               Guerrieri Guerriero della Terra 2019   
4  161857655                      Château Voigny Sauternes 2019   
5  15609827

This query selects 10 wines from the toplists ranking based on rank and price < 40 euros     
These wines should be good to promote and raise sales.

In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import streamlit as st
# Connect to SQLite database
conn = sqlite3.connect(r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\vivino.db')

# Create a cursor object
cur = conn.cursor()

cur.execute("""
    SELECT
        v.name,
        vintage_toplists_rankings.rank,
        toplists.name,
        v.price_euros
    FROM
        vintages v
        JOIN vintage_toplists_rankings ON v.id = vintage_toplists_rankings.vintage_id
        JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
    WHERE v.price_euros < 20000
    GROUP BY
        toplists.name
    ORDER BY
        vintage_toplists_rankings.rank
    LIMIT 100;
""")
# fetch the query result
query_result = cur.fetchall()

# create a dataframe from the query result
df = pd.DataFrame(query_result, columns=['vintage_name', 'vintage_rank', 'toplists_name', 'vintages.price_euros'])
print(df.info)
# close the database connection
conn.close()

# create a scatter plot using plotly express
scatter_plot = px.scatter(df, x='toplists_name', y='vintage_rank', 
                          color='vintage_rank', color_continuous_scale='viridis',
                          size='vintage_rank', hover_name='vintage_name', title='wine ratings')
scatter_plot.show()
# Save the DataFrame to a CSV file in the "data" directory
csv_file_path = r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\wines_to_highlight_by_toplists.csv'
df.to_csv(csv_file_path, index=False)
#add streamlit code
st.plotly_chart(scatter_plot)

                                         vintage_name  vintage_rank  \
0               Arzuaga Reserva Ribera del Duero 2018             1   
1                      Pago de Carraovejas Tinto 2020             1   
2    Tommasi Amarone della Valpolicella Classico 2018             1   
3   Buena Vista Chateau Buena Vista Cabernet Sauvi...             1   
4                                Masseto Toscana 2005             1   
..                                                ...           ...   
95  Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...             7   
96  Château Pavie Macquin Saint-Émilion Grand Cru ...             7   
97                            Vilafonté Series M 2014             7   
98                           Vallone Graticciaia 2012             7   
99  Casa Ermelinda Freitas Dona Ermelinda Grande R...             8   

                                        toplists_name  vintages.price_euros  
0                              Bestsellers in Córdoba                

2023-09-01 09:22:52.327 
  command:

    streamlit run c:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()